<a href="https://colab.research.google.com/github/foreverchnf/GLOUNNLP-Bert/blob/master/bert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

查看colab系统信息

In [2]:
!sudo lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic


安装符合colab系统的cuda

In [3]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda


--2020-05-25 08:21:42--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]      56  --.-KB/s    in 0s      

2020-05-25 08:21:42 (1.71 MB/s) - ‘cuda-ubuntu1804.pin’ saved [56/56]

--2020-05-25 08:21:50--  http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:80... connected.
HTTP request sent, awaiting response... 20

In [4]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.2.89


安装mxnet和gluonnlp

mxnet版本（1.5.x不兼容）
更新到1.6.0


In [0]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python

In [0]:
!pip install --upgrade gluonnlp

导入bert预训练的模型

使用bert large（bert_24_1024_16）或bert base（bert_12_768_12）

In [1]:
import gluonnlp as nlp; import mxnet as mx;
model, vocab = nlp.model.get_model('bert_24_1024_16', dataset_name='book_corpus_wiki_en_uncased', use_classifier=False, use_decoder=False);
tokenizer = nlp.data.BERTTokenizer(vocab, lower=True);
transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=512, pair=False, pad=False);
sample = transform(['Hello world!']);
words, valid_len, segments = mx.nd.array([sample[0]]), mx.nd.array([sample[1]]), mx.nd.array([sample[2]]);
seq_encoding, cls_encoding = model(words, segments, valid_len);

Vocab file is not found. Downloading.


查看模型

In [2]:
print(model)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=1024)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(1024 -> 1024, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(1024 -> 4096, linear)
          (activation): GELU()
          (ffn_2): Dense(4096 -> 1024, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=1024)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=1024)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAt

查看当前CPU和GPU信息

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1571769956481967850, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6806001383688610304
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14998909871614939945
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15888542575432864728
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

进入目标文件夹

In [6]:
import os
os.chdir("/content/drive/My Drive/sentence_embedding/bert")
!ls

bert_qa_evaluate.py	finetune_squad.py  pretraining_utils.py
data			fp16_utils.py	   __pycache__
embedding.py		index.rst	   run.ipynb
export.py		__init__.py	   run_pretraining.py
finetune_classifier.py	model		   utils.py


使用gpu进行fine-tuning

对不同task的命令使用可查阅https://gluon-nlp.mxnet.io/model_zoo/bert/index.html

In [7]:
!python3 finetune_classifier.py --gpu 0 --task_name SST --batch_size 32 --seed 2 --lr 2e-5 --warmup_ratio 0.1 --epochs 5 --log_interval 400


INFO:root:08:34:53 Namespace(accumulate=None, batch_size=32, bert_dataset='book_corpus_wiki_en_uncased', bert_model='bert_12_768_12', calib_mode='customize', deploy=False, dev_batch_size=8, dtype='float32', early_stop=None, epochs=5, epsilon=1e-06, gpu=0, log_interval=400, lr=2e-05, max_len=128, model_parameters=None, model_prefix=None, num_calib_batches=5, only_calibration=False, only_inference=False, optimizer='bertadam', output_dir='./output_dir', pretrained_bert_parameters=None, quantized_dtype='auto', round_to=None, seed=2, task_name='SST', training_steps=None, warmup_ratio=0.1)
INFO:root:08:34:53 Model file not found. Downloading to /root/.mxnet/models/bert_12_768_12_book_corpus_wiki_en_uncased-75cc780f.params.
INFO:root:08:35:30 processing dataset...
INFO:root:08:35:38 Now we are doing BERT classification training on gpu(0)!
INFO:root:08:35:38 training steps=10523
INFO:root:08:36:28 [Epoch 1 Batch 400/2108] loss=0.4201, lr=0.0000076, metrics:accuracy:0.7970
INFO:root:08:37:18 [E